## Python Blackjack
For this project you will make a Blackjack game using Python. Click <a href="http://www.hitorstand.net/strategy.php">here</a> to familiarize yourself with the the rules of the game. You won't be implementing every rule "down to the letter" with the game, but we will doing a simpler version of the game. This assignment will be given to further test your knowledge on object-oriented programming concepts.

### Rules:

`1. ` The game will have two players: the Dealer and the Player. The game will start off with a deck of 52 cards. The 52 cards will consist of 4 different suits: Clubs, Diamonds, Hearts and Spades. For each suit, there will be cards numbered 1 through 13. <br>
**Note: No wildcards will be used in the program**

`2. ` When the game begins, the dealer will shuffle the deck of cards, making them randomized. After the dealer shuffles, it will deal the player 2 cards and will deal itself 2 cards from. The Player should be able to see both of their own cards, but should only be able to see one of the Dealer's cards.
 
`3. ` The objective of the game is for the Player to count their cards after they're dealt. If they're not satisfied with the number, they have the ability to 'Hit'. A hit allows the dealer to deal the Player one additional card. The Player can hit as many times as they'd like as long as they don't 'Bust'. A bust is when the Player is dealt cards that total more than 21.

`4. ` If the dealer deals the Player cards equal to 21 on the **first** deal, the Player wins. This is referred to as Blackjack. Blackjack is **NOT** the same as getting cards that equal up to 21 after the first deal. Blackjack can only be attained on the first deal.

`5. ` The Player will never see the Dealer's hand until the Player chooses to 'stand'. A Stand is when the player tells the dealer to not deal it anymore cards. Once the player chooses to Stand, the Player and the Dealer will compare their hands. Whoever has the higher number wins. Keep in mind that the Dealer can also bust. 

TODO: 
    - Finish implementing game class
    - Fix calculate_hand to check for aces (1 or 11) and royals
    - Fix print total for dealer to print only the shown cards

In [36]:
from random import shuffle
# Contains all the requirements for a game
class Game():
    def __init__(self):
        self.deck = None
        self.dealer = Dealer()
        self.player = Player()
        self.start()
    
    def start(self):
        print('Welcome to BlackJack!')
        while True:
            self.deck = Deck().deck
            
            # Allows player to add money to their balance
            if self.player.balance == 0:
                ask = input('Would you like to add more money to your balance?(y/n): ').lower()
                
                if ask == 'y':
                    self.player.add_to_balance()
                elif ask == 'n':
                    print('Thank you for playing!')
                    break
                else:
                    print('Please enter "y" for yes and "n" for no.')
            
            # Ensures player cannot enter a bet amount higher than their balance
            bet_amount = self.player.bet()
            while bet_amount == -1:
                bet_amount = self.player.bet()
            
            # Deals cards to player and dealer
            self.deal_card('dealer')
            self.deal_card('player')
            self.deal_card('dealer')
            self.deal_card('player')

            while True:
                self.dealer.show_cards()
                self.player.show_cards()
                print('*'*50)

                ask = input('Would you like to hit(h), stand(s) or add to your balance(b)?: ').lower()
                
                # Player hits
                if ask == 'h':
                    self.deal_card('player')
                    
                    if self.is_bust('player'):
                        self.dealer.show_cards(True)
                        self.player.show_cards()
                        print('You have busted!')
                        self.player.calculate_balance('lose', bet_amount)
                        break
                # Player stands
                elif ask == 's':
                    self.dealer.show_cards(True)
                    self.player.show_cards()
                    self.stand(bet_amount)
                    break
                # Player adds to balance
                elif ask == 'b':
                    self.player.add_to_balance()
                
                else:
                    print('Please enter "h" for hit, "s" for stand or "b" to add to balance')
                    
            self.dealer.clear_hand()
            self.player.clear_hand()
            ask = input('Would you like to play again?(Y/N): ')
            # Ends the game
            if ask.lower() == 'n':
                break
    
    def deal_card(self, person_type):
        if person_type == 'player':
            self.player.cards.append(self.deck.pop())
        else:
            self.dealer.cards.append(self.deck.pop())
    
    # Checks if dealer/player has busted
    def is_bust(self, person_type):
        self.player.calculate_hand()
        self.dealer.calculate_hand()
        
        # Checks if the player busted
        if person_type == 'player':
            return True if self.player.hand_total > 21 else False
        # Checks if the dealer busted
        else:
            return True if self.dealer.hand_total > 21 else False
    
    
    def stand(self, bet_amount):
        self.dealer.calculate_hand()
        busted = False
        
        # Forces dealer to hit until 17 or higher
        while self.dealer.hand_total < 17:
            print('Dealer hits...')
            self.deal_card('dealer')
            self.dealer.show_cards(True)
            self.player.show_cards()
            
            if self.is_bust('dealer'):
                print('Dealer has busted!')
                busted = True
                break
        
        if busted:
            status = 'win'
        else:
            status = self.check_win()
        
        self.player.calculate_balance(status, bet_amount)
    
    # Verifies who won
    def check_win(self):
        self.player.calculate_hand()
        self.dealer.calculate_hand()
        
        # Dealer won
        if self.dealer.hand_total > self.player.hand_total:
            print('Dealer has won!')
            return 'lose'
        # Tied
        elif self.dealer.hand_total == self.player.hand_total:
            print("It's a tie!")
            return 'tie'
        # Player won
        else:
            print('Player has won!')
            return 'win'

class Person():
    def __init__(self, person_type):
        self.cards = []
        self.person_type = person_type.lower()
        self.hand_total = 0
    
    # Calculates the sum of player/dealer's hand
    def calculate_hand(self):
        self.hand_total = sum([value if value not in [11, 12, 13] else 10 for value, suit in self.cards])
    
    # Prints player and dealers cards
    def show_cards(self, end_of_game=False):
        # Formats the print string with the first card
        print_cards = f'{self.person_type.title()} has {self.convert_card(self.cards[0])}'
        for i in range(1, len(self.cards)):
            print_cards += ' - '
            # prints all the player cards or the deaelers card when the game is over
            if i == 1 and self.person_type == 'dealer':
                print_cards += 'HIDDEN '
            elif self.person_type == 'player' or (self.person_type == 'dealer' and end_of_game):
                print_cards += self.convert_card(self.cards[i])
                
        
        # Attaches hand total to the end of the print statement
        self.calculate_hand()
        print_cards += f' - Total: {self.hand_total}'
            
        print(print_cards)
    
    # Converts Ace and Royals to print accordingly instead of their numeric values
    def convert_card(self, card):
        if card[0] not in [1, 11, 12, 13]:
            return f'{card[0]} of {card[1]}'
        elif card[0] == 1:
            return "Ace of " + card[1]
        elif card[0] == 11:
            return "Jack of " + card[1]
        elif card[0] == 12:
            return "Queen of " + card[1]
        elif card[0] == 13:
            return "King of " + card[1]
        
    # Clears dealer and player's hands
    def clear_hand(self):
        self.cards = []
            
class Dealer(Person):
    def __init__(self):
        super().__init__('dealer')
            
class Player(Person):
    def __init__(self, balance=100):
        super().__init__('player')
        self.balance = balance
    
    # Prints the current player balance
    def print_balance(self):
        print('Your current balance is', self.balance)
    
    # Prompts user to enter a bet
    def bet(self):
        self.print_balance()
        bet_amount = int(input('Please enter your bet: '))
        if bet_amount > self.balance or not isinstance(bet_amount, int):
            print('Invalid bet, please try again.')
            return -1
        else:
            return bet_amount
    
    # Calculates new player balance depending on winning status
    def calculate_balance(self, status, bet_amount):
        if status.lower() == 'win':
            self.balance += bet_amount
        elif status.lower() == 'lose':
            self.balance -= bet_amount
        
        self.print_balance()
    
    # Adds specified amount to player's balance
    def add_to_balance(self):
        money = int(input('Please enter the amount you would like to add: '))
        self.balance += money
        
class Deck():
    # Generates a deck when a new instance is created
    def __init__(self):
        self.deck = self.generate_deck()
        self.shuffle_deck()
    
    # Generates a new deck of 52 cards
    def generate_deck(self):
        suits = ['Diamonds', 'Hearts', 'Spades', 'Clubs']
        values = [i for i in range(1, 14)]
        return [(v, s) for v in values for s in suits]
    
    # Shuffles the instance deck
    def shuffle_deck(self):
        shuffle(self.deck)

In [37]:
game = Game()

Welcome to BlackJack!
Your current balance is 100
Please enter your bet: 10
Dealer has 9 of Clubs - HIDDEN  - Total: 19
Player has King of Spades - 2 of Clubs - Total: 12
**************************************************
Would you like to hit(h), stand(s) or add to your balance(b)?: h
Dealer has 9 of Clubs - HIDDEN  - Total: 19
Player has King of Spades - 2 of Clubs - 3 of Diamonds - Total: 15
**************************************************
Would you like to hit(h), stand(s) or add to your balance(b)?: h
Dealer has 9 of Clubs - HIDDEN  - Total: 19
Player has King of Spades - 2 of Clubs - 3 of Diamonds - 10 of Spades - Total: 25
You have busted!
Your current balance is 90
Would you like to play again?(Y/N): n


In [ ]:
deck = Deck()

In [ ]:
from random import shuffle
num = [1,2,3,4,5]
shuffle(num)
num

In [ ]:
game.dealer.calculate_hand()
game.dealer.hand_total